# Imports

In [35]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance 


# Input

In [36]:
transformation = Transformation()

user_number_triangles = 2500
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [37]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [38]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [39]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [40]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.61092671 0.61128215 0.6109267  ... 0.56875219 0.82559015 0.56875219]
[0.61796984 0.61832938 0.61796984 ... 0.57530911 0.83510806 0.57530911]
[0.67742743 0.67752175 0.67742743 ... 0.66613443 0.73158603 0.66613443]
(5999,)


### Multinomial Sampling

In [41]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 1 0 ... 0 0 0]


In [42]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(list(graph._node.keys()))[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[75.0252    , -0.66553295,  3.8       ],
       [75.01418   , -0.49935842, 16.7       ],
       [75.        ,  0.        ,  3.8       ],
       [75.00158   ,  0.16653232, 16.7       ],
       [75.0252    ,  0.66553295,  3.8       ]], dtype=float32)

# KNN extended graph

In [43]:
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 45259


# Edge Predictor

In [44]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(5999, 64)

In [45]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [46]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [47]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [48]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 30)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  (1, 10)	1
  :	:
  (5997, 5840)	1
  (5997, 5843)	1
  (5997, 5844)	1
  (5997, 5847)	1
  (5997, 5983)	1
  (5997, 5985)	1
  (5997, 5987)	1
  (5997, 5989)	1
  (5997, 5991)	1
  (5997, 5993)	1
  (5997, 5995)	1
  (5998, 5849)	1
  (5998, 5851)	1
  (5998, 5853)	1
  (5998, 5855)	1
  (5998, 5858)	1
  (5998, 5859)	1
  (5998, 5862)	1
  (5998, 5984)	1
  (5998, 5986)	1
  (5998, 5988)	1
  (5998, 5990)	1
  (5998, 5992)	1
  (5998, 5994)	1
  (5998, 5996)	1
[[0.06730613 0.0666828  0.06730613 ... 0.07112957 0.07112957 0.07112957]
 [1.06736763 0.07141639 0.07094569 ... 0.06806554 0.06806554 0.06806554]
 [1.01242179 0.48449237 0.06730613 ... 0.07112957 0.07112957 0.07112957]
 ...
 [1.12403398 1.05240169 1.12403398 ... 0.07142857 0.07142857 0.

In [49]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [50]:
print(A_s)  # symmétrique
print(A_s.shape)

[[0.00212065 0.00218701 0.00212739 ... 0.04404889 0.04380777 0.04401605]
 [0.00218701 0.0022581  0.00219628 ... 0.04523508 0.04500435 0.04520366]
 [0.00212739 0.00219628 0.00213811 ... 0.04417192 0.0439308  0.04413908]
 ...
 [0.04404889 0.04523508 0.04417192 ... 0.96190574 0.95742707 0.96156531]
 [0.04380777 0.04500435 0.0439308  ... 0.95742707 0.95595376 0.95739438]
 [0.04401605 0.04520366 0.04413908 ... 0.96156531 0.95739438 0.96130633]]
(5999, 5999)


# Face Classifier

### TriConv

#### Inputs

In [51]:
nodes = list(extended_graph.nodes())
edges = list(extended_graph.edges())

igraph_g = igraphGraph(directed=extended_graph.is_directed())

igraph_g.add_vertices(nodes)
igraph_g.add_edges([(nodes.index(u), nodes.index(v)) for u, v in edges])
print(igraph_g.summary())

IGRAPH UN-- 5999 45259 -- 
+ attr: name (v)


In [52]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[  37 4206 4207]
 [4191 4192 4193]
 [4176 4177 4178]]
126401


In [53]:
triangles = np.array(igraph_g.vs['name'])[triangles_ids_igraph]
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[75.01686     0.66553295  3.7812707 ]
  [75.01482     0.66553295  3.7796104 ]
  [75.01586     0.66553295  3.7804136 ]]

 [[75.05917     1.3272538   3.7185588 ]
  [75.063354    1.3272538   3.7217672 ]
  [75.06736     1.3272538   3.72519   ]]

 [[75.13281     1.9813722   3.6171947 ]
  [75.142204    1.9813722   3.624396  ]
  [75.1512      1.9813722   3.6320791 ]]]
126401


In [54]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(igraph_g.vs))
p_init[i] = (A_s_ij + A_s_ik + A_s_jk) / 3

#### Calculate barycenter

In [55]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(126401, 3)
[[75.01585     0.66553295  3.7804317 ]
 [75.06329     1.3272538   3.7218387 ]
 [75.142075    1.9813722   3.6245565 ]]


#### KNN Tri

In [56]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [57]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(126401, 19, 3)


#### calculate e norm matrix

In [58]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(126401, 3)
[[0.00263391 0.00131664 0.00131821]
 [0.00527009 0.01053509 0.00526864]]


#### Calculate r

In [59]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(126401,)

In [60]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(126401, 19)
(126401, 19)


In [61]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(126401, 19, 5)


#### Calculate f

In [62]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.999999999999999
(5999,)
[0.00012462 0.00012462 0.00012462 ... 0.00012438 0.00012438 0.00012438]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_6656\1047055924.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))


# Simplified Mesh

In [63]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[:1])

(1999, 3, 3)
[[[78.921715 -8.535009 91.41643 ]
  [79.45103  -8.535009 91.19718 ]
  [79.96814  -8.535009 90.95053 ]]]


In [64]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 1108
Number of edges: 3064


In [66]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [67]:
keys_input = list(graph._node.keys())
print("Les points avant K-NN : ", keys_input)

Les points avant K-NN :  [(75.0252, -0.66553295, 3.8), (75.01418, -0.49935842, 16.7), (75.0, 0.0, 3.8), (75.00158, 0.16653232, 16.7), (75.0252, 0.66553295, 3.8), (75.03937, 0.8314692, 16.7), (75.10067, 1.3272538, 3.8), (75.12734, 1.4916434, 16.7), (75.22596, 1.9813722, 3.8), (75.26499, 2.1432736, 16.7), (75.40037, 2.6241417, 3.8), (75.45153, 2.7826276, 16.7), (75.62288, 3.2518802, 3.8), (75.68588, 3.4060426, 16.7), (75.892235, 3.8609922, 3.8), (75.96672, 4.0099483, 16.7), (76.20689, 4.4479885, 3.8), (76.29242, 4.590885, 16.7), (76.565025, 5.0095077, 3.8), (76.661125, 5.145526, 16.7), (76.96461, 5.5423326, 3.8), (77.07072, 5.670694, 16.7), (77.40334, 6.0434117, 3.8), (77.51886, 6.1633806, 16.7), (77.87872, 6.5098743, 3.8), (78.002975, 6.620764, 16.7), (78.388016, 6.9390492, 3.8), (78.5203, 7.0402236, 16.7), (78.92831, 7.328478, 3.8), (79.06786, 7.4193583, 16.7), (79.4965, 7.6759295, 3.8), (79.64253, 7.755995, 16.7), (80.08935, 7.9794145, 3.8), (80.24101, 8.048206, 16.7), (80.703445, 8.2

In [68]:
print("dim points avant K-NN : ", len(keys_input))  #return 5999

dim points avant K-NN :  5999


Récupération des points de l'espace de départ Ps

In [69]:
keys_output = list(extended_graph)
print("sampled points : ", keys_output)

sampled points :  [(75.0252, -0.66553295, 3.8), (75.02517, -0.66553295, 3.798681), (75.02506, -0.66553295, 3.7973657), (75.024895, -0.66553295, 3.7960575), (75.02465, -0.66553295, 3.79476), (75.024345, -0.66553295, 3.793477), (75.02397, -0.66553295, 3.792212), (75.02353, -0.66553295, 3.7909682), (75.023026, -0.66553295, 3.7897491), (75.02245, -0.66553295, 3.7885582), (75.02183, -0.66553295, 3.7873986), (75.02113, -0.66553295, 3.7862735), (75.020386, -0.66553295, 3.785186), (75.019585, -0.66553295, 3.7841394), (75.01873, -0.66553295, 3.7831361), (75.01418, -0.49935842, 16.7), (75.07711, -1.1623888, 16.7), (75.00158, 0.16653232, 16.7), (75.19, -1.8187612, 16.7), (75.03937, 0.8314692, 16.7), (75.35221, -2.464716, 16.7), (75.12734, 1.4916434, 16.7), (75.562805, -3.0965528, 16.7), (75.26499, 2.1432736, 16.7), (75.82059, -3.710653, 16.7), (75.45153, 2.7826276, 16.7), (76.12407, -4.303499, 16.7), (75.68588, 3.4060426, 16.7), (76.47152, -4.871695, 16.7), (75.96672, 4.0099483, 16.7), (75.0, 0.0

In [70]:
print("dim sample points : ", len(keys_output))  #return: 50

dim sample points :  5999


Distance Matrix

In [71]:
def squared_euclidean_distances(x, y):
    distances = np.zeros((len(y), len(x)))

    for i, y_point in enumerate(y):
        for j, x_point in enumerate(x):
            distances[i, j] = np.sum((np.array(y_point) - np.array(x_point))**2)

    return distances

distances_matrix = squared_euclidean_distances(keys_input, keys_output)
print("Squared Euclidean Distances Matrix Shape:", distances_matrix.shape)
print("Distances Matrix:\n", distances_matrix)




Squared Euclidean Distances Matrix Shape: (5999, 5999)
Distances Matrix:
 [[0.00000000e+00 1.66437759e+02 4.43569124e-01 ... 7.55992676e+02
  1.46491833e+03 7.44313416e+02]
 [1.74051183e-06 1.66471786e+02 4.43569332e-01 ... 7.55962769e+02
  1.46488940e+03 7.44285217e+02]
 [6.95832659e-06 1.66505737e+02 4.43569183e-01 ... 7.55929321e+02
  1.46485510e+03 7.44253479e+02]
 ...
 [8.30382812e+02 1.38115601e+03 8.37244019e+02 ... 2.12411728e+01
  1.71544479e+02 2.75444756e+01]
 [1.54158472e+03 2.07931128e+03 1.54842505e+03 ... 1.59703339e+02
  2.12411747e+01 1.65241180e+02]
 [8.20979858e+02 1.35897095e+03 8.28425049e+02 ... 1.57033443e+01
  1.65241180e+02 2.12411747e+01]]


In [72]:
# Calculer le min des x de distances_matrix
def min_pour_x_list(distances_matrix, x_in):
    return np.min(distances_matrix[:, x_in])

# Calculer le min des y de distances_matrix
def min_pour_y_list(distances_matrix, y_out):
    return np.min(distances_matrix[y_out, :])


In [73]:
def d_P_Ps(keys_x, keys_y, p_y):
    """
    Calculate the Probabilistic Chamfer distance between two sets of points.

    Parameters:
    - keys_x (list): The input vertex set, a list of tuples.
    - keys_y (list): The sampled points, a list of tuples.
    - p_y (list): Their respective probabilities, a weight associated with each point in keys_y.

    Returns:
        chamfer_distance (float): The Probabilistic Chamfer distance between the two sets of points.
    """
    min_for_x = [p_y[i] * min_pour_x_list(distances_matrix, i) for i in range(len(keys_x))]
    min_for_y = [p_y[j] * min_pour_y_list(distances_matrix, j) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(min_for_y)
    sum_for_x = np.sum(min_for_x)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance


In [74]:
print("PROBA : ", normalized_inclusion_score)

PROBA :  [0.00020681 0.00020684 0.00020681 ... 0.00020336 0.00022335 0.00020336]


In [75]:
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Chamfer Distance:", prob_chamfer_dist)

Chamfer Distance: 0.0


## Probabilistic Surfaces Distance

barycentres b et b_hat

In [76]:
b = np.array(list(graph._node.keys()))
print("Barycentre b: ", b)

Barycentre b:  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]


In [77]:
b_hat = np.array(list(extended_graph._node.keys()))
print("B_HAT : ", b_hat)

B_HAT :  [[ 75.0252      -0.66553295   3.8       ]
 [ 75.02517     -0.66553295   3.798681  ]
 [ 75.02506     -0.66553295   3.7973657 ]
 ...
 [ 51.          -6.3966537  -11.043411  ]
 [ 39.          -6.8353896  -10.542333  ]
 [ 51.          -6.8353896  -10.542333  ]]


Matrice de distances après le KNN

In [78]:
def matrix_distances_after_knn(b_hat, number_neigh=3):
    """
    Calculate distances between barycenters after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """

    # Connect the extended graph using the k-NN process
    extended_graph = connect_extended_graph(b_hat, number_neigh)

    # Extract barycenters after the k-NN process
    b_hat_extended = np.array(list(extended_graph._node.keys()))

    # Calculate distances after k-NN
    distances_after = cdist(b_hat_extended, b_hat_extended)

    return distances_after

# Example usage with b_hat
distances_after = matrix_distances_after_knn(b_hat)

# Print the results
print("Distances After k-NN:\n", distances_after)

NameError: name 'connect_extended_graph' is not defined

In [ ]:
def probabilistic_surface_distance_extended(b_i, b_hat_i, p_b_hat_i):
    """
    Calculate Probabilistic Surfaces Distance 

    Parameters:
    - b: array-like, shape (N, 3)
        Barycenters of the ground truth surface.
    - b_hat: array-like, shape (M, 3)
        Barycenters of the generated triangles.
    - p_b_hat: array-like, shape (M,)
        Probabilities corresponding to the generated triangles.

    Returns:
    - d_S_Ss: float
        Probabilistic Surfaces Distance.
    """
    # Debugging print statements
    print("b shape:", b_i.shape)
    print("b_hat shape:", b_hat_i.shape)
    print("p_b_hat shape:", p_b_hat_i.shape)

    p_b_hat_i = normalized_inclusion_score.reshape((-1, 1))
    # Calculate Probabilistic Surfaces Distance
    d_S_Ss = np.sum(p_b_hat_i * np.min(np.sum((b_hat_i[:, np.newaxis] - b_i)**2, axis=2), axis=1))

    print("Intermediate results:")
    print("Squared differences shape:", np.sum((b_hat_i[:, np.newaxis] - b_i)**2, axis=2).shape)
    print("Min values shape:", np.min(np.sum((b_hat_i[:, np.newaxis] - b_i)**2, axis=2), axis=1).shape)

    return d_S_Ss

# Debugging: Call the function and print intermediate values
result_distance = probabilistic_surface_distance_extended(b, b_hat, normalized_inclusion_score)
print("Probabilistic Surfaces Distance:", result_distance)

## Triangle Collision Loss

In [ ]:
def triangle_collision_loss(generated_triangles, probabilities):
    """
    Calculate triangle collision loss to penalize triangles that penetrate each other.

    Parameters:
    - generated_triangles (numpy.ndarray): Array representing the generated triangles.
    - probabilities (numpy.ndarray): Array of probabilities for each generated triangle.

    Returns:
    - float: Triangle collision loss.
    """
    num_triangles = len(generated_triangles)

    # Create a 3D array to represent all combinations of triangle pairs
    triangle_pairs_i, triangle_pairs_j = np.meshgrid(range(num_triangles), range(num_triangles), indexing='ij')

    # Filter out pairs where i is not equal to j
    valid_pairs_mask = triangle_pairs_i != triangle_pairs_j

    # Extract triangles and probabilities for each pair
    triangles_i = generated_triangles[triangle_pairs_i[valid_pairs_mask]]
    triangles_j = generated_triangles[triangle_pairs_j[valid_pairs_mask]]
    probs_i = probabilities[triangle_pairs_i[valid_pairs_mask]]

    # Calculate the plane normals and distances for all triangles
    plane_normals = np.cross(triangles_i[:, 1] - triangles_i[:, 0], triangles_i[:, 2] - triangles_i[:, 0])

    # Calculate the edge vectors for all edges of triangles_j
    edge_vectors = triangles_j[:, [1, 2, 0], :] - triangles_j[:, [0, 1, 2], :]

    # Calculate the dot product of edge vectors and plane normals
    dot_products = np.sum(np.cross(edge_vectors, plane_normals[:, np.newaxis, :]) * triangles_i[:, 0], axis=2)

    # Count the number of faces penetrated for each triangle
    faces_penetrated = np.sum(dot_products < 0, axis=1)

    # Calculate the loss for each triangle
    loss_per_triangle = probs_i * faces_penetrated

    # Sum up the losses and normalize by the number of triangles
    triangle_losses = np.sum(loss_per_triangle) / num_triangles

    return triangle_losses

